In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://developer.download.nvidia.com/compute/c

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-27 00:51:46--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.65MB/s    in 0.2s    

2022-02-27 00:51:46 (5.65 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles

# Access Amazon reviews for Pet products:
reviews_url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
filename = reviews_url.split("/")[len(reviews_url.split("/"))-1]
spark.sparkContext.addFile(reviews_url)
start_df = spark.read.csv(SparkFiles.get(filename), sep="\t", header=True, inferSchema=True).dropna(how="any")
start_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

# Determine number of reviews

In [5]:
# Count the rows in DataFrame:
num_reviews = start_df.count()
print(f"This data set contains {num_reviews:,} reviews.")

This data set contains 2,643,241 reviews.


# Cleaned up DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date

# Create the review_id DataFrame:
review_id_df = start_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")]).drop_duplicates(["review_id"])
review_id_df.printSchema()
review_id_df.show()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100065P6TTS3J|   42711834|B000HHM6PS|      64793786| 2014-04-06|
|R10007MH6NTVFM|   25105396|B0090Z9FFC|     675354291| 2015-06-24|
|R1000CIZTRNP23|   25423435|B00K1B6RCI|     308737701| 2015-03-06|
|R1000JOVLD0J41|   34245087|B000TZ7022|     870738517| 2010-11-13|
|R1000YCFR347I2|   48610961|B00CO527IC|     935823254| 2014-09-05|
|R1000ZYX4P32OD|   27895609|B002DXCQMM|     184150150| 2015-02-27|
|R10015VI9KRLUW|   18810197|B005IZOB5M|     259423137| 2014-03-27|
|R1001KC9GI6ARJ|   11232277|B0080YGXPE|     551772997| 2015-04-16|
|R1001RSHOZYRB6|   25116815|B003TWKJQQ|   

In [14]:
# Create the products DataFrame:
products_df = start_df.select(["product_id", "product_title"]).drop_duplicates(["product_id"])
products_df.printSchema()
products_df.show()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0310824230|Advantage Flea Co...|
|0615553605|Pet Qwerks Treat ...|
|0684836483|250 Things You Ca...|
|0763004227|Golden Retriever ...|
|0764102885|Saint Bernards (B...|
|0764118269|  The Finch Handbook|
|0764144294|             Gerbils|
|0876051468|The Stray Cat Han...|
|0972585419|Feathered Phonics...|
|0975412817|The Notes Pocket ...|
|0977264815|ABC's of Behavior...|
|0982083513|Piece of Pie A La...|
|0982393555|Pill Wrap Paste -...|
|0983794804|Pathway Dog Journ...|
|0983794812|Pathway Cat Journ...|
|1083812550|Lees Wooden Air D...|
|1083813215|Lees Under Gravel...|
|1300347635|      Pet Herbal FPS|
|1300450991|Puppy Booster Her...|
|1300451041|Kitten Booster Ca...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the customers DataFrame:
customers_df = start_df.groupBy("customer_id").count().withColumnRenamed("count", "customer_count")
customers_df.printSchema()
customers_df.show()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10270641|             1|
|   18365872|             1|
|   16711087|             1|
|   10742726|             2|
|   41169638|             1|
|   43622307|             1|
|   24540309|             2|
|   28258386|             1|
|   35329257|             2|
|   14552054|             1|
|   14529507|             5|
|   45392827|             5|
|   47282953|             1|
|    8201930|             1|
|   20109760|             2|
|   16405801|             4|
|   15056685|            21|
|   20840575|             2|
|   39048303|             1|
|    5596610|             1|
+-----------+--------------+
only showing top 20 rows



In [9]:
# select columns from vine_df
# Create the vine DataFrame:
vine_df = start_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.printSchema()
vine_df.show()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| REAKC26P07MDN|          5|            0|          0|   N|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|
|R14QJW3XF8QO1P|          5|            0|          0|   N|
|R2HB7AX0394ZGY|          5|            0|          0|   N|
| RGKMPDQGSAHR3|          5|            0|          0|   N|
|R1DJCVPQGCV66E|          5|            0|          0|   N|
|R3V52EAWLPBFQG|          3|            0|          0|   N|
|R3DKO8J1J28QBI|          2|            0|          0|   N|
| R764DBXGRNECG|          5|            1|          1|   N|
| RW1853GAT0Z9F|          5|            0|          0|   N|
|R3

# Push to AWS RDS instance

In [10]:
# Set endpoint, database name, and password:
endpoint = ""
database = ""
password = ""

In [17]:
# set up properties to connect to AWS Postgres
mode = "overwrite"
jdbc_url = f"jdbc:postgresql://{endpoint}:5432/{database}"
config = {"user": "postgres", "password": password, "driver": "org.postgresql.Driver"}

In [19]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [18]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [20]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [21]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)